In [7]:
import pandas as pd
import numpy as np
import json
import ast


df1 = pd.read_csv('Kickstarter001.csv')

photo_link = []
country = []
state = []
town = []
loc_type = []
project_url = []
reward_url =[]
list_category = []
list_subcategory = []

for i in range(len(df1)):
    photo_dict =df1.iloc[i].photo
    photo_link.append(ast.literal_eval(photo_dict)['full'])
    
for i in range(len(df1)):
    try:
        category, subcategory  = json.loads(df1.iloc[i].category)['slug'].split('/')
        list_category.append(category)
        list_subcategory.append(subcategory)
    except Exception as e:
        list_category.append(None)
        list_subcategory.append(None)

for i in range(len(df1)):
    try:
        loc_dict = json.loads(df1.iloc[i].location)
        country.append(str(loc_dict['country']))
    except Exception as e:
        country.append("None")
        
for i in range(len(df1)):
    try:
        loc_dict = json.loads(df1.iloc[i].location)
        state.append(str(loc_dict['state']))
    except Exception as e:
        state.append("None")
for i in range(len(df1)):
    try:
        loc_dict = json.loads(df1.iloc[i].location)
        town.append(str(loc_dict['name']))
    except Exception as e:
        town.append("None")
for i in range(len(df1)):
    try:
        loc_dict = json.loads(df1.iloc[i].location)
        loc_type.append(str(loc_dict['type']))
    except Exception as e:
        loc_type.append("None")

for i in range(len(df1)):
    try:
        project_url.append(str(json.loads(df1.iloc[i]['urls'])['web']['project']))
    except Exception as e:
        project_url.append("None")
for i in range(len(df1)):
    try:
        reward_url.append(str(json.loads(df1.iloc[i]['urls'])['web']['rewards']))
    except Exception as e:
        reward_url.append("None")        


df1['clean_category'] = np.array(list_category)
df1['subcategory'] = np.array(list_subcategory)
df1['photo_link'] = np.array(photo_link)
df1['clean_country'] = np.array(country)
df1['state'] = np.array(state)
df1['town'] = np.array(town)
df1['location_type'] = np.array(loc_type)
df1['project_url'] = project_url
df1['reward_url'] = reward_url

df1.drop(['category','urls','photo','location','profile', 'creator',
'currency_symbol',
'currency_trailing_code',
'current_currency',
'disable_communication', 
'fx_rate',
'friends',
'is_backing',
'is_starred'],axis=1,inplace=True)


,backers_count,blurb,converted_pledged_amount,country,created_at,currency,deadline,goal,id,is_starrable,...,usd_type,permissions,clean_category,subcategory,photo_link,clean_country,town,location_type,project_url,reward_url
0,22,"See some cool illustrations, Help make them av...",634,US,1361602103,USD,1367651035,600.0,1082237073,False,...,domestic,NaN,art,illustration,https://ksr-ugc.imgix.net/assets/011/485/371/d...,US,Lockport,Town,https://www.kickstarter.com/projects/690850738...,https://www.kickstarter.com/projects/690850738...
1,28,"High-quality 24""x36"" print parodying the ill-f...",1016,US,1364919898,USD,1367283600,1000.0,1574516491,False,...,domestic,NaN,art,illustration,https://ksr-ugc.imgix.net/assets/011/502/866/8...,US,Nashville,Town,https://www.kickstarter.com/projects/164500077...,https://www.kickstarter.com/projects/164500077...
2,8,This is a collection of 5 iconic New Orleans i...,945,US,1364070331,USD,1368040821,300.0,2080857761,False,...,domestic,NaN,art,illustration,https://ksr-ugc.imgix.net/assets/011/498/271/f...,US,New Orleans,Town,https://www.kickstarter.com/projects/174428992...,https://www.kickstarter.com/projects/174428992...
3,125,Award winning author and illustrator Shane W. ...,5949,US,1364365408,USD,1368881098,5000.0,802310873,False,...,domestic,NaN,art,illustration,https://ksr-ugc.imgix.net/assets/011/499/967/0...,US,Kansas City,Town,https://www.kickstarter.com/projects/olusdream...,https://www.kickstarter.com/projects/olusdream...
4,68,We'll illustrate you as a zombie and place you...,5539,GB,1366639145,GBP,1369699038,1200.0,1222458071,False,...,domestic,NaN,art,illustration,https://ksr-ugc.imgix.net/assets/011/511/971/7...,GB,North Yorkshire,County,https://www.kickstarter.com/projects/100515158...,https://www.kickstarter.com/projects/100515158...


In [8]:
df1.to_csv('cleaned_data001.csv')

In [17]:
from bs4 import BeautifulSoup # documentation available at : #www.crummy.com/software/BeautifulSoup/bs4/doc/
from BeautifulSoup import NavigableString, Tag
import requests # To send http requests and access the page : docs.python-requests.org/en/latest/
import csv # To create the output csv file
import unicodedata # To work with the string encoding of the data
cleaned_df = pd.read_csv('cleaned_data001.csv')
projects_url = cleaned_df['project_url'].tolist()
list_description =[]
list_rewards = []
    
for url in projects_url:
    try:
        r = requests.get(url, timeout = 10) #Sending a request to access the page
    except Exception,e:
        break
    data = r.text
    soup = BeautifulSoup(data) # Getting the page source into the soup
    ps = []
    for div in soup.find_all('div'):
        if (div.get('class') != None and div.get('class')[0] == 'full-description'):
            ps = div.find_all('p')
        if (div.get('class') != None and div.get('class')[0] == 'NS_projects__rewards_list'):
            li_len = len(div.find_all('li')) -1
        desc = ' ' 
        for i in ps:
            desc+= i.text
    list_rewards.append(li_len)
    list_description.append(desc.encode('utf-8'))


KeyboardInterrupt: 

In [ ]:
cleaned_df['Description'] = np.array(list_description)
cleaned_df['Rewards'] = np.array(list_rewards)

In [18]:
len(list_description
   )

2340

In [19]:
len(list_rewards)

2340

In [21]:
the_file = open('descriptions.txt', 'w')
for item in list_description:
    the_file.write("%s\n" % item)

In [23]:
cleaned_panel = df1[0:2340]

In [24]:
cleaned_panel['Description'] = np.array(list_description)

/Users/saurabh/venv/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [25]:
cleaned_panel['Rewards'] = np.array(list_rewards)

/Users/saurabh/venv/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [26]:
cleaned_panel.to_csv('Cleaned_data_with_desc_and_rewards.csv')